# Chapter 5: Classes and Interfaces

---

This chapter covers best practices for designing classes and interfaces in Python, including composition patterns, polymorphism, inheritance, and advanced attribute handling.

## Item 37: Compose Classes Instead of Nesting Built-in Types

### The Problem with Deep Nesting

Dictionaries, lists, tuples, and sets are easy to use, but there's a danger of **overextending them** to write brittle code. When your bookkeeping gets complicated, break it into classes.

### Example: Grade Tracking System Evolution

#### Version 1: Simple Dictionary

In [ ]:
# Simple gradebook with overall grades
class SimpleGradebook:
    def __init__(self):
        self._grades = {}
    
    def add_student(self, name):
        self._grades[name] = []
    
    def report_grade(self, name, grade):
        self._grades[name].append(grade)
    
    def average_grade(self, name):
        grades = self._grades[name]
        return sum(grades) / len(grades)

# Usage
book = SimpleGradebook()
book.add_student('Isaac Newton')
book.report_grade('Isaac Newton', 90)
book.report_grade('Isaac Newton', 95)
book.report_grade('Isaac Newton', 85)

print(f"Average: {book.average_grade('Isaac Newton')}")

#### Version 2: Adding Subjects (Nested Dictionary)

In [ ]:
from collections import defaultdict

class BySubjectGradebook:
    def __init__(self):
        self._grades = {}  # Outer dict
    
    def add_student(self, name):
        self._grades[name] = defaultdict(list)  # Inner dict
    
    def report_grade(self, name, subject, grade):
        by_subject = self._grades[name]
        grade_list = by_subject[subject]
        grade_list.append(grade)
    
    def average_grade(self, name):
        by_subject = self._grades[name]
        total, count = 0, 0
        for grades in by_subject.values():
            total += sum(grades)
            count += len(grades)
        return total / count

# Usage
book = BySubjectGradebook()
book.add_student('Albert Einstein')
book.report_grade('Albert Einstein', 'Math', 75)
book.report_grade('Albert Einstein', 'Math', 65)
book.report_grade('Albert Einstein', 'Gym', 90)
book.report_grade('Albert Einstein', 'Gym', 95)

print(f"Average: {book.average_grade('Albert Einstein')}")

#### Version 3: Adding Weights (Tuple in List) - TOO COMPLEX!

In [ ]:
# WARNING: This is getting too complicated!
class WeightedGradebook:
    def __init__(self):
        self._grades = {}
    
    def add_student(self, name):
        self._grades[name] = defaultdict(list)
    
    def report_grade(self, name, subject, score, weight):
        by_subject = self._grades[name]
        grade_list = by_subject[subject]
        grade_list.append((score, weight))  # Tuple inside list
    
    def average_grade(self, name):
        by_subject = self._grades[name]
        score_sum, score_count = 0, 0
        
        for subject, scores in by_subject.items():
            subject_avg, total_weight = 0, 0
            for score, weight in scores:
                subject_avg += score * weight
                total_weight += weight
            
            score_sum += subject_avg / total_weight
            score_count += 1
        
        return score_sum / score_count

# Usage - unclear what the numbers mean!
book = WeightedGradebook()
book.add_student('Albert Einstein')
book.report_grade('Albert Einstein', 'Math', 75, 0.05)
book.report_grade('Albert Einstein', 'Math', 65, 0.15)
book.report_grade('Albert Einstein', 'Math', 70, 0.80)
book.report_grade('Albert Einstein', 'Gym', 100, 0.40)
book.report_grade('Albert Einstein', 'Gym', 85, 0.60)

print(f"Average: {book.average_grade('Albert Einstein')}")

### 🚨 When to Refactor to Classes

**Avoid nesting beyond one level:**
- Dictionaries containing dictionaries → Hard to read
- Long tuples → Positional confusion
- Multiple layers of built-in types → Maintenance nightmare

**Time to refactor when:**
1. You're nesting dictionaries inside dictionaries
2. Tuples grow beyond 2 items
3. Code becomes unclear to other programmers

### Refactored Solution: Using Classes

#### Step 1: Create Grade Class Using namedtuple

In [ ]:
from collections import namedtuple

# Simple, immutable data container
Grade = namedtuple('Grade', ('score', 'weight'))

# Test it
grade = Grade(score=95, weight=0.45)
print(f"Score: {grade.score}, Weight: {grade.weight}")

# Calculate weighted average
grades = []
grades.append(Grade(95, 0.45))
grades.append(Grade(85, 0.55))

total = sum(score * weight for score, weight in grades)
total_weight = sum(weight for _, weight in grades)
average_grade = total / total_weight

print(f"Weighted average: {average_grade}")

#### ⚠️ Limitations of namedtuple

1. **No default arguments** - Can't specify optional parameters
2. **Still accessible by index** - Can lead to bugs in APIs
3. **For many attributes** - Consider `dataclasses` module instead

#### Step 2: Create Subject Class

In [ ]:
class Subject:
    def __init__(self):
        self._grades = []
    
    def report_grade(self, score, weight):
        self._grades.append(Grade(score, weight))
    
    def average_grade(self):
        total, total_weight = 0, 0
        for grade in self._grades:
            total += grade.score * grade.weight
            total_weight += grade.weight
        return total / total_weight

# Test it
math = Subject()
math.report_grade(75, 0.05)
math.report_grade(65, 0.15)
math.report_grade(70, 0.80)
print(f"Math average: {math.average_grade()}")

#### Step 3: Create Student Class

In [ ]:
class Student:
    def __init__(self):
        self._subjects = defaultdict(Subject)
    
    def get_subject(self, name):
        return self._subjects[name]
    
    def average_grade(self):
        total, count = 0, 0
        for subject in self._subjects.values():
            total += subject.average_grade()
            count += 1
        return total / count

# Test it
albert = Student()
math = albert.get_subject('Math')
math.report_grade(75, 0.05)
math.report_grade(65, 0.15)
math.report_grade(70, 0.80)

gym = albert.get_subject('Gym')
gym.report_grade(100, 0.40)
gym.report_grade(85, 0.60)

print(f"Overall average: {albert.average_grade()}")

#### Step 4: Create Gradebook Class

In [ ]:
class Gradebook:
    def __init__(self):
        self._students = defaultdict(Student)
    
    def get_student(self, name):
        return self._students[name]

# Final usage - Much clearer!
book = Gradebook()
albert = book.get_student('Albert Einstein')

math = albert.get_subject('Math')
math.report_grade(75, 0.05)
math.report_grade(65, 0.15)
math.report_grade(70, 0.80)

gym = albert.get_subject('Gym')
gym.report_grade(100, 0.40)
gym.report_grade(85, 0.60)

print(f"Albert's average: {albert.average_grade()}")

### 💡 Key Takeaways

✅ **Avoid** dictionaries with nested dictionaries  
✅ **Use** `namedtuple` for lightweight, immutable data  
✅ **Refactor** to multiple classes when internal state gets complicated  
✅ Line count may increase, but **readability** and **maintainability** improve dramatically

### 🎯 Practice Exercise

Try extending the `Grade` class to include a notes field from the teacher:

In [ ]:
# Exercise: Extend Grade to support optional notes
# Hint: namedtuple might not be the best choice anymore

# Your code here:

---

## Item 38: Accept Functions Instead of Classes for Simple Interfaces

### Functions as First-Class Objects

Python makes it easy to pass functions as arguments because **functions are first-class objects**. This is ideal for **hooks** and **callbacks**.

### Example 1: Simple Function Hook

In [ ]:
# Sorting with a function hook
names = ['Socrates', 'Archimedes', 'Plato', 'Aristotle']
names.sort(key=len)  # Function passed as hook
print(names)

### Example 2: defaultdict with Function Hook

In [ ]:
from collections import defaultdict

def log_missing():
    print('Key added')
    return 0

# Using function as default factory
current = {'green': 12, 'blue': 3}
increments = [
    ('red', 5),
    ('blue', 17),
    ('orange', 9),
]

result = defaultdict(log_missing, current)
print('Before:', dict(result))

for key, amount in increments:
    result[key] += amount

print('After:', dict(result))

### Example 3: Stateful Closure

In [ ]:
def increment_with_report(current, increments):
    added_count = 0
    
    def missing():
        nonlocal added_count  # Stateful closure
        added_count += 1
        return 0
    
    result = defaultdict(missing, current)
    for key, amount in increments:
        result[key] += amount
    
    return result, added_count

# Usage
current = {'green': 12, 'blue': 3}
increments = [('red', 5), ('blue', 17), ('orange', 9)]

result, count = increment_with_report(current, increments)
print(f"Result: {dict(result)}")
print(f"Added {count} keys")

### Example 4: Class with State (Better Approach)

In [ ]:
class CountMissing:
    def __init__(self):
        self.added = 0
    
    def missing(self):
        self.added += 1
        return 0

# Usage with method reference
counter = CountMissing()
result = defaultdict(counter.missing, current)  # Method reference

for key, amount in increments:
    result[key] += amount

print(f"Added {counter.added} keys")

### Example 5: Using `__call__` (Best Approach)

In [ ]:
class BetterCountMissing:
    def __init__(self):
        self.added = 0
    
    def __call__(self):
        self.added += 1
        return 0

# Usage - instance itself is callable
counter = BetterCountMissing()
print(f"Is callable? {callable(counter)}")

result = defaultdict(counter, current)  # Cleaner!
for key, amount in increments:
    result[key] += amount

print(f"Added {counter.added} keys")

### 📊 Comparison: When to Use What

| Approach | When to Use | Clarity | Statefulness |
|----------|-------------|---------|-------------|
| **Simple Function** | No state needed | ⭐⭐⭐⭐⭐ | ❌ |
| **Closure** | Hidden state | ⭐⭐ | ✅ |
| **Class + Method** | Clear state | ⭐⭐⭐ | ✅ |
| **Class + `__call__`** | Stateful callable | ⭐⭐⭐⭐⭐ | ✅ |

### 💡 Key Takeaways

✅ Functions are **first-class objects** in Python  
✅ Use **simple functions** for stateless hooks  
✅ Use **`__call__`** method for stateful callables  
✅ `__call__` makes class purpose clear as a function-like interface

---

## Item 39: Use @classmethod Polymorphism to Construct Objects Generically

### The Problem: Object Construction in Hierarchies

Python only supports **one constructor** (`__init__`) per class. How do we create objects generically across different subclasses?

### Example: MapReduce Implementation

#### Step 1: Define Abstract Base Classes

In [ ]:
# Input data abstraction
class InputData:
    def read(self):
        raise NotImplementedError

# Concrete implementation: read from file
class PathInputData(InputData):
    def __init__(self, path):
        super().__init__()
        self.path = path
    
    def read(self):
        with open(self.path) as f:
            return f.read()

In [ ]:
# Worker abstraction
class Worker:
    def __init__(self, input_data):
        self.input_data = input_data
        self.result = None
    
    def map(self):
        raise NotImplementedError
    
    def reduce(self, other):
        raise NotImplementedError

# Concrete implementation: line counter
class LineCountWorker(Worker):
    def map(self):
        data = self.input_data.read()
        self.result = data.count('\n')
    
    def reduce(self, other):
        self.result += other.result

#### Step 2: Manual Construction (Inflexible)

In [ ]:
import os

def generate_inputs(data_dir):
    for name in os.listdir(data_dir):
        yield PathInputData(os.path.join(data_dir, name))

def create_workers(input_list):
    workers = []
    for input_data in input_list:
        workers.append(LineCountWorker(input_data))
    return workers

# Problem: Not generic! Hardcoded to PathInputData and LineCountWorker

### ❌ The Problem

If we create new `InputData` or `Worker` subclasses, we must rewrite:
- `generate_inputs()`
- `create_workers()`
- `mapreduce()`

**Not maintainable!**

### ✅ Solution: Class Method Polymorphism

In [ ]:
# Generic base class with @classmethod constructor
class GenericInputData:
    def read(self):
        raise NotImplementedError
    
    @classmethod
    def generate_inputs(cls, config):
        raise NotImplementedError

# Concrete implementation
class PathInputData(GenericInputData):
    def __init__(self, path):
        super().__init__()
        self.path = path
    
    def read(self):
        with open(self.path) as f:
            return f.read()
    
    @classmethod
    def generate_inputs(cls, config):
        data_dir = config['data_dir']
        for name in os.listdir(data_dir):
            yield cls(os.path.join(data_dir, name))  # Generic!

In [ ]:
# Generic Worker with @classmethod constructor
class GenericWorker:
    def __init__(self, input_data):
        self.input_data = input_data
        self.result = None
    
    def map(self):
        raise NotImplementedError
    
    def reduce(self, other):
        raise NotImplementedError
    
    @classmethod
    def create_workers(cls, input_class, config):
        workers = []
        for input_data in input_class.generate_inputs(config):
            workers.append(cls(input_data))  # Generic!
        return workers

# Concrete implementation
class LineCountWorker(GenericWorker):
    def map(self):
        data = self.input_data.read()
        self.result = data.count('\n')
    
    def reduce(self, other):
        self.result += other.result

In [ ]:
# Generic MapReduce function
from threading import Thread

def execute(workers):
    threads = [Thread(target=w.map) for w in workers]
    for thread in threads:
        thread.start()
    for thread in threads:
        thread.join()
    
    first, *rest = workers
    for worker in rest:
        first.reduce(worker)
    return first.result

def mapreduce(worker_class, input_class, config):
    workers = worker_class.create_workers(input_class, config)
    return execute(workers)

In [ ]:
# Test setup
import os
import random

def write_test_files(tmpdir):
    os.makedirs(tmpdir, exist_ok=True)
    for i in range(100):
        with open(os.path.join(tmpdir, str(i)), 'w') as f:
            f.write('\n' * random.randint(0, 100))

tmpdir = 'test_inputs'
write_test_files(tmpdir)

# Now fully generic!
config = {'data_dir': tmpdir}
result = mapreduce(LineCountWorker, PathInputData, config)
print(f'There are {result} lines')

### 💡 Key Takeaways

✅ Python only supports **one `__init__`** per class  
✅ Use **`@classmethod`** to define alternative constructors  
✅ Class method polymorphism enables **generic object construction**  
✅ Eliminates need to rewrite glue code for new subclasses

---

## Item 40: Initialize Parent Classes with super

### The Old Way (Direct Call) - ❌ Problematic

In [ ]:
class MyBaseClass:
    def __init__(self, value):
        self.value = value

class MyChildClass(MyBaseClass):
    def __init__(self):
        MyBaseClass.__init__(self, 5)  # Direct call - works but brittle

child = MyChildClass()
print(f"Value: {child.value}")

### Problem 1: Multiple Inheritance Order Confusion

In [ ]:
class TimesTwo:
    def __init__(self):
        self.value *= 2

class PlusFive:
    def __init__(self):
        self.value += 5

# Parent order: MyBaseClass, TimesTwo, PlusFive
class OneWay(MyBaseClass, TimesTwo, PlusFive):
    def __init__(self, value):
        MyBaseClass.__init__(self, value)
        TimesTwo.__init__(self)
        PlusFive.__init__(self)

foo = OneWay(5)
print(f'First ordering value is (5 * 2) + 5 = {foo.value}')

# Different parent order but same __init__ calls!
class AnotherWay(MyBaseClass, PlusFive, TimesTwo):
    def __init__(self, value):
        MyBaseClass.__init__(self, value)
        TimesTwo.__init__(self)  # Same order as OneWay
        PlusFive.__init__(self)

bar = AnotherWay(5)
print(f'Second ordering value is {bar.value}')  # Same result - confusing!

### Problem 2: Diamond Inheritance

In [ ]:
# Diamond inheritance pattern
#       MyBaseClass
#      /           \
# TimesSeven    PlusNine
#      \           /
#       ThisWay

class TimesSeven(MyBaseClass):
    def __init__(self, value):
        MyBaseClass.__init__(self, value)
        self.value *= 7

class PlusNine(MyBaseClass):
    def __init__(self, value):
        MyBaseClass.__init__(self, value)
        self.value += 9

class ThisWay(TimesSeven, PlusNine):
    def __init__(self, value):
        TimesSeven.__init__(self, value)
        PlusNine.__init__(self, value)  # Resets value!

foo = ThisWay(5)
print(f'Should be (5 * 7) + 9 = 44 but is {foo.value}')  # Wrong!

### ✅ Solution: Use `super()`

In [ ]:
class TimesSevenCorrect(MyBaseClass):
    def __init__(self, value):
        super().__init__(value)  # Python handles MRO
        self.value *= 7

class PlusNineCorrect(MyBaseClass):
    def __init__(self, value):
        super().__init__(value)
        self.value += 9

class GoodWay(TimesSevenCorrect, PlusNineCorrect):
    def __init__(self, value):
        super().__init__(value)

foo = GoodWay(5)
print(f'Should be 7 * (5 + 9) = 98 and is {foo.value}')  # Correct!

### Understanding Method Resolution Order (MRO)

In [ ]:
# Python uses C3 linearization algorithm
mro_str = '\n'.join(repr(cls) for cls in GoodWay.mro())
print("Method Resolution Order:")
print(mro_str)

# Order of execution:
# 1. GoodWay.__init__
# 2. TimesSevenCorrect.__init__
# 3. PlusNineCorrect.__init__
# 4. MyBaseClass.__init__
# Then work happens in reverse order!

### Different Ways to Call `super()`

In [ ]:
# All three are equivalent:

class ExplicitTrisect(MyBaseClass):
    def __init__(self, value):
        super(ExplicitTrisect, self).__init__(value)
        self.value /= 3

class AutomaticTrisect(MyBaseClass):
    def __init__(self, value):
        super(__class__, self).__init__(value)
        self.value /= 3

class ImplicitTrisect(MyBaseClass):
    def __init__(self, value):
        super().__init__(value)  # ⭐ Preferred!
        self.value /= 3

# All produce the same result
assert ExplicitTrisect(9).value == 3
assert AutomaticTrisect(9).value == 3
assert ImplicitTrisect(9).value == 3
print("All three methods work!")

### 💡 Key Takeaways

✅ Always use **`super()`** instead of direct `__init__` calls  
✅ Python's **MRO** (C3 linearization) handles diamond inheritance  
✅ Prefer **`super()`** with zero arguments (most maintainable)  
✅ Only provide parameters to `super()` when wrapping/reusing specific functionality

---

## Item 41: Consider Composing Functionality with Mix-in Classes

### What is a Mix-in?

A **mix-in** is a class that:
- Defines a **small set of additional methods**
- Has **no instance attributes**
- Doesn't require **`__init__`** to be called
- Provides **generic functionality** that can be applied to many classes

### Example 1: ToDictMixin

In [ ]:
class ToDictMixin:
    def to_dict(self):
        return self._traverse_dict(self.__dict__)
    
    def _traverse_dict(self, instance_dict):
        output = {}
        for key, value in instance_dict.items():
            output[key] = self._traverse(key, value)
        return output
    
    def _traverse(self, key, value):
        if isinstance(value, ToDictMixin):
            return value.to_dict()
        elif isinstance(value, dict):
            return self._traverse_dict(value)
        elif isinstance(value, list):
            return [self._traverse(key, i) for i in value]
        elif hasattr(value, '__dict__'):
            return self._traverse_dict(value.__dict__)
        else:
            return value

In [ ]:
# Use the mix-in
class BinaryTree(ToDictMixin):
    def __init__(self, value, left=None, right=None):
        self.value = value
        self.left = left
        self.right = right

# Create a tree
tree = BinaryTree(10,
    left=BinaryTree(7, right=BinaryTree(9)),
    right=BinaryTree(13, left=BinaryTree(11)))

# Convert to dict automatically!
import json
print(json.dumps(tree.to_dict(), indent=2))

### Overriding Mix-in Behavior

In [ ]:
# Tree with parent reference (would cause cycles)
class BinaryTreeWithParent(BinaryTree):
    def __init__(self, value, left=None, right=None, parent=None):
        super().__init__(value, left=left, right=right)
        self.parent = parent
    
    def _traverse(self, key, value):
        if isinstance(value, BinaryTreeWithParent) and key == 'parent':
            return value.value  # Prevent cycles
        else:
            return super()._traverse(key, value)

# Test it
root = BinaryTreeWithParent(10)
root.left = BinaryTreeWithParent(7, parent=root)
root.left.right = BinaryTreeWithParent(9, parent=root.left)

print(json.dumps(root.to_dict(), indent=2))

### Example 2: Composable Mix-ins

In [ ]:
import json

class JsonMixin:
    @classmethod
    def from_json(cls, data):
        kwargs = json.loads(data)
        return cls(**kwargs)
    
    def to_json(self):
        return json.dumps(self.to_dict())

# Compose multiple mix-ins
class DatacenterRack(ToDictMixin, JsonMixin):
    def __init__(self, switch=None, machines=None):
        self.switch = Switch(**switch)
        self.machines = [Machine(**kwargs) for kwargs in machines]

class Switch(ToDictMixin, JsonMixin):
    def __init__(self, ports=None, speed=None):
        self.ports = ports
        self.speed = speed

class Machine(ToDictMixin, JsonMixin):
    def __init__(self, cores=None, ram=None, disk=None):
        self.cores = cores
        self.ram = ram
        self.disk = disk

In [ ]:
# Test serialization
serialized = """{
    "switch": {"ports": 5, "speed": 1e9},
    "machines": [
        {"cores": 8, "ram": 32e9, "disk": 5e12},
        {"cores": 4, "ram": 16e9, "disk": 1e12}
    ]
}"""

deserialized = DatacenterRack.from_json(serialized)
roundtrip = deserialized.to_json()

print("Original and roundtrip match:",
      json.loads(serialized) == json.loads(roundtrip))

### 💡 Key Takeaways

✅ Mix-ins avoid multiple inheritance complexity  
✅ No instance attributes or `__init__` required  
✅ Can be **composed** to build complex functionality  
✅ Can be **overridden** for custom behavior  
✅ Works with both **instance methods** and **class methods**

---

## Item 42: Prefer Public Attributes Over Private Ones

### Python's Visibility Types

Python has only **two** types of attribute visibility:
1. **Public**: `self.public_field`
2. **Private**: `self.__private_field`

In [ ]:
class MyObject:
    def __init__(self):
        self.public_field = 5
        self.__private_field = 10
    
    def get_private_field(self):
        return self.__private_field

foo = MyObject()
print(f"Public: {foo.public_field}")
print(f"Private (via method): {foo.get_private_field()}")

# Direct access to private fails
try:
    print(foo.__private_field)
except AttributeError as e:
    print(f"Error: {e}")

### How Private Attributes Actually Work

In [ ]:
# Python transforms private attribute names
print("Object's __dict__:")
print(foo.__dict__)

# You can still access it!
print(f"\nAccessing 'private' field: {foo._MyObject__private_field}")

### Problem with Private Attributes in Subclasses

In [ ]:
class MyParentObject:
    def __init__(self):
        self.__private_field = 71

class MyChildObject(MyParentObject):
    def get_private_field(self):
        return self.__private_field  # Won't work!

baz = MyChildObject()
try:
    baz.get_private_field()
except AttributeError as e:
    print(f"Error: {e}")

### The Right Way: Protected Attributes

In [ ]:
class MyStringClass:
    def __init__(self, value):
        # Protected by convention (single underscore)
        self._value = value
    
    def get_value(self):
        return str(self._value)

class MyIntegerSubclass(MyStringClass):
    def get_value(self):
        return int(self._value)  # Works!

foo = MyIntegerSubclass(5)
print(f"Integer value: {foo.get_value()}")

### When to Use Private Attributes

**Only** use private attributes (`__`) to avoid **naming conflicts** in public APIs:

In [ ]:
# Public API class (many unknown subclasses)
class ApiClass:
    def __init__(self):
        self.__value = 5  # Avoid conflicts
    
    def get(self):
        return self.__value

class Child(ApiClass):
    def __init__(self):
        super().__init__()
        self._value = 'hello'  # No conflict!

a = Child()
print(f"{a.get()} and {a._value} are different")

### 📊 Naming Conventions Summary

| Style | Usage | Meaning |
|-------|-------|----------|
| `public_field` | Normal use | ✅ Accessible to everyone |
| `_protected_field` | Internal API | ⚠️ Use with caution |
| `__private_field` | Rare | ⛔ Avoid naming conflicts only |

### 💡 Key Takeaways

✅ **Document** protected fields instead of forcing privacy  
✅ Use **single underscore** (`_protected`) for internal APIs  
✅ **Rarely** use double underscore (only for public API naming conflicts)  
✅ Remember: **"We are all consenting adults here"**

---

## Item 43: Inherit from collections.abc for Custom Container Types

### The Problem with Custom Containers

In [ ]:
# Simple custom list with frequency counting
class FrequencyList(list):
    def __init__(self, members):
        super().__init__(members)
    
    def frequency(self):
        counts = {}
        for item in self:
            counts[item] = counts.get(item, 0) + 1
        return counts

# Works great!
foo = FrequencyList(['a', 'b', 'a', 'c', 'b', 'a', 'd'])
print(f"Length: {len(foo)}")
foo.pop()
print(f"After pop: {foo}")
print(f"Frequency: {foo.frequency()}")

### Custom Sequence Without Inheriting from list

In [ ]:
# Binary tree node
class BinaryNode:
    def __init__(self, value, left=None, right=None):
        self.value = value
        self.left = left
        self.right = right

# Make it indexable
class IndexableNode(BinaryNode):
    def _traverse(self):
        if self.left is not None:
            yield from self.left._traverse()
        yield self
        if self.right is not None:
            yield from self.right._traverse()
    
    def __getitem__(self, index):
        for i, item in enumerate(self._traverse()):
            if i == index:
                return item.value
        raise IndexError(f'Index {index} is out of range')

In [ ]:
# Build a tree
tree = IndexableNode(
    10,
    left=IndexableNode(
        5,
        left=IndexableNode(2),
        right=IndexableNode(6, right=IndexableNode(7))
    ),
    right=IndexableNode(15, left=IndexableNode(11))
)

# Can index it!
print(f"Index 0: {tree[0]}")
print(f"Index 1: {tree[1]}")
print(f"11 in tree? {11 in tree}")
print(f"Tree as list: {list(tree)}")

# But len() doesn't work!
try:
    print(len(tree))
except TypeError as e:
    print(f"Error: {e}")

### Adding `__len__` Method

In [ ]:
class SequenceNode(IndexableNode):
    def __len__(self):
        for count, _ in enumerate(self._traverse(), 1):
            pass
        return count

tree = SequenceNode(
    10,
    left=SequenceNode(
        5,
        left=SequenceNode(2),
        right=SequenceNode(6, right=SequenceNode(7))
    ),
    right=SequenceNode(15, left=SequenceNode(11))
)

print(f"Tree length: {len(tree)}")

# But still missing count() and index()!
try:
    tree.count(10)
except AttributeError as e:
    print(f"Missing method: {e}")

### ✅ Solution: Use collections.abc

In [ ]:
from collections.abc import Sequence

# Validates required methods
class BadType(Sequence):
    pass

try:
    foo = BadType()
except TypeError as e:
    print(f"Error: {e}")

In [ ]:
# Provides free methods!
class BetterNode(SequenceNode, Sequence):
    pass

tree = BetterNode(
    10,
    left=BetterNode(
        5,
        left=BetterNode(2),
        right=BetterNode(6, right=BetterNode(7))
    ),
    right=BetterNode(15, left=BetterNode(11))
)

# Now we get index() and count() for free!
print(f"Index of 7: {tree.index(7)}")
print(f"Count of 10: {tree.count(10)}")

### 📋 Common collections.abc Classes

| Abstract Base Class | Required Methods | Free Methods |
|---------------------|------------------|---------------|
| **Sequence** | `__getitem__`, `__len__` | `index`, `count`, `__contains__`, `__iter__`, `__reversed__` |
| **MutableSequence** | + `__setitem__`, `__delitem__`, `insert` | + `append`, `reverse`, `extend`, `pop`, `remove` |
| **Set** | `__contains__`, `__iter__`, `__len__` | `<=`, `<`, `==`, `!=`, `>`, `>=`, `&`, `\|`, `-`, `^` |
| **MutableSet** | + `add`, `discard` | + `clear`, `pop`, `remove`, `__ior__`, `__iand__`, `__ixor__`, `__isub__` |
| **Mapping** | `__getitem__`, `__iter__`, `__len__` | `get`, `keys`, `items`, `values`, `__contains__` |
| **MutableMapping** | + `__setitem__`, `__delitem__` | + `pop`, `popitem`, `clear`, `update`, `setdefault` |

### 💡 Key Takeaways

✅ Inherit from **`list` or `dict`** for simple use cases  
✅ Use **`collections.abc`** for custom container types  
✅ Automatic **validation** of required methods  
✅ Get many **free methods** automatically  
✅ Ensures **correct interface** and behaviors

---

## 🎯 Chapter 5 Summary

### Best Practices Covered:

1. **Item 37**: Compose classes instead of nesting built-in types
2. **Item 38**: Use functions for simple interfaces, `__call__` for stateful ones
3. **Item 39**: Use `@classmethod` for generic object construction
4. **Item 40**: Always use `super()` to initialize parent classes
5. **Item 41**: Use mix-ins to compose functionality
6. **Item 42**: Prefer public/protected attributes over private
7. **Item 43**: Inherit from `collections.abc` for custom containers

### Key Principles:

✅ **Composition over Complexity**: Break nested structures into classes  
✅ **Polymorphism**: Leverage class and instance method polymorphism  
✅ **Proper Inheritance**: Use `super()` and understand MRO  
✅ **Flexibility**: Keep code extensible with public APIs  
✅ **Standards**: Use built-in abstract base classes